In [434]:
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

import tensorflow as tf

In [483]:
train = pd.read_csv('train.csv')
#=======
test = pd.read_csv('test.csv')

In [484]:
train_y = train[['SalePrice']]

In [485]:
train.shape

(1460, 81)

In [486]:
test.shape

(1459, 80)

In [487]:
def del_columns(train):
    cols = train.columns
        
    size = len(cols)
    
    for i in range(size):
        #处理的都是非数值型的
        if train[cols[i]].dtype == np.object:
            _ = train[cols[i]].describe(include = 'O')
            
            #print(cols[i] , end=',')
            
            if _.freq > 900:
                train = train.drop(cols[i] , axis=1)
                #print(cols[i] , _.freq)
            else:
                null_count = train[cols[i]].isnull()
                
                if null_count.sum() > 1000:
                    train = train.drop(cols[i] , axis=1)
                    #print('nan:', cols[i] , _.freq)

    return train

In [488]:
train = train.drop(['Id' , 'SalePrice'] , axis = 1)
test = test.drop(['Id'] , axis = 1)

In [489]:
#将一些用不上的feature 直接删除
train = del_columns(train)
#==========
test = del_columns(test)

In [490]:
#train['MasVnrType'].value_counts()
train = train.drop(['MasVnrType'] , axis = 1)
#上面的也需要删除 太多空None
#==========
test = test.drop(['MasVnrType'] , axis = 1)


In [491]:
test = test.drop(['ExterQual'] , axis = 1)

In [492]:
print(len(train.columns))
#=============
print(len(test.columns))

48
48


In [493]:
train['BsmtFinSF2'].value_counts()
train = train.drop(['BsmtFinSF2'] , axis = 1)
train = train.drop(['LowQualFinSF'] , axis = 1)
train = train.drop(['BsmtHalfBath'] , axis = 1)
train = train.drop(['KitchenAbvGr'] , axis = 1)
train = train.drop(['EnclosedPorch'] , axis = 1)
train = train.drop(['3SsnPorch'] , axis = 1)
train = train.drop(['PoolArea'] , axis = 1)
train = train.drop(['MiscVal'] , axis = 1)
train = train.drop(['ScreenPorch'] , axis = 1)

#删除 0太多
#================
test['BsmtFinSF2'].value_counts()
test = test.drop(['BsmtFinSF2'] , axis = 1)
test = test.drop(['LowQualFinSF'] , axis = 1)
test = test.drop(['BsmtHalfBath'] , axis = 1)
test = test.drop(['KitchenAbvGr'] , axis = 1)
test = test.drop(['EnclosedPorch'] , axis = 1)
test = test.drop(['3SsnPorch'] , axis = 1)
test = test.drop(['PoolArea'] , axis = 1)
test = test.drop(['MiscVal'] , axis = 1)
test = test.drop(['ScreenPorch'] , axis = 1)

In [494]:
print(len(train.columns))
print(len(test.columns))

39
39


In [495]:
def detect_nan_int_float(train):
    #检测一下 int and float中有缺失值的feature
    cols = train.columns
    size = len(cols)
    
    nan_cols = []
    
    for i in range(size):
        if train[cols[i]].dtype == np.int64 or train[cols[i]].dtype == np.float64:
            _ = train[cols[i]].isnull()
            
            if _.sum() != 0:
                nan_cols.append(cols[i])
    
    return nan_cols

In [496]:
nan_cols = detect_nan_int_float(train)
#==========
nan_cols_test = detect_nan_int_float(test)

In [497]:
print(nan_cols)
#数值类型feature中含有缺失值的feature
#年份也是用平均值来填充 符合大众化口味
#=========
print(nan_cols_test)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']


In [498]:
#含有缺失值的feature（数值型feature）
#GarageYrBlt 这是年份信息

def fillna_num(nan_cols , train):
    size = len(nan_cols)
    
    for i in range(size):
        train[nan_cols[i]] = train[nan_cols[i]].fillna(train[nan_cols[i]].median())
    
    return train

In [499]:
train = fillna_num(nan_cols , train)
#==========
test = fillna_num(nan_cols_test , test)

In [500]:
nan_cols = detect_nan_int_float(train)
#==========
nan_cols_test = detect_nan_int_float(test)
#缺失值全部填充完成
print(nan_cols)
#=========
print(nan_cols_test)

[]
[]


In [501]:
def classier(train):
    cols = train.columns
    size = len(cols)
    
    obj_cols = []
    
    for i in range(size):
        if train[cols[i]].dtype == np.object:
            obj_cols.append(cols[i])
    
    return obj_cols

In [502]:
obj_cols = classier(train)
#============
obj_cols_test = classier(test)

In [503]:
#非数字的feature
print(obj_cols)
#==============
print(obj_cols_test)

['Neighborhood', 'HouseStyle', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtQual', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish']
['Neighborhood', 'HouseStyle', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtQual', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish']


In [504]:
def nan_obj(obj_cols , train):
    size = len(obj_cols)
    nan_obj_cols = []
    
    for i in range(size):
        if train[obj_cols[i]].isnull().sum() != 0:
            nan_obj_cols.append(obj_cols[i])
    
    return nan_obj_cols

In [505]:
nan_obj_cols = nan_obj(obj_cols , train)
#===========
nan_obj_cols_test = nan_obj(obj_cols_test , test)

In [506]:
#这些是具有缺失值 非数值型feature
print(nan_obj_cols)
#===========
print(nan_obj_cols_test)

['BsmtQual', 'BsmtFinType1', 'FireplaceQu', 'GarageType', 'GarageFinish']
['Exterior1st', 'Exterior2nd', 'BsmtQual', 'BsmtFinType1', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish']


In [507]:
#查看频率次数最高的 和 缺失值数量
def look(nan_obj_cols , train):
    size = len(nan_obj_cols)
    
    for i in range(size):
        _ = train[nan_obj_cols[i]].describe()
        
        print(nan_obj_cols[i] , _.top , _.freq , train[nan_obj_cols[i]].isnull().sum())

In [508]:
print(look(nan_obj_cols , train))
print(look(nan_obj_cols_test , test))

BsmtQual TA 649 37
BsmtFinType1 Unf 430 37
FireplaceQu Gd 380 690
GarageType Attchd 870 81
GarageFinish Unf 605 81
None
Exterior1st VinylSd 510 1
Exterior2nd VinylSd 510 1
BsmtQual TA 634 44
BsmtFinType1 GLQ 431 42
KitchenQual TA 757 1
FireplaceQu Gd 364 730
GarageType Attchd 853 76
GarageFinish Unf 625 78
None


In [509]:
train['FireplaceQu'].value_counts().index

Index(['Gd', 'TA', 'Fa', 'Ex', 'Po'], dtype='object')

In [510]:
def get_map(index):
    map_dict = {}
    
    [map_dict.setdefault(y , x+1) for x,y in enumerate(index)]

    return map_dict

def o2n(obj_cols , train):
    size = len(obj_cols)
    
    for i in range(size):
        map_dict = get_map(train[obj_cols[i]].value_counts().index)
        train[obj_cols[i]] = train[obj_cols[i]].map(map_dict)
        
    return train

In [511]:
train = o2n(obj_cols , train)
#======
test = o2n(obj_cols_test , test)

In [512]:
#进行 缺失值填充  直接使用数字0来填充
#这里是对非数值型feature的缺失值
def fillna_obj(nan_obj_cols , train):
    size = len(nan_obj_cols)
    
    for i in range(size):
        train[nan_obj_cols[i]] = train[nan_obj_cols[i]].fillna(0)
    
    return train

In [513]:
train = fillna_obj(nan_obj_cols , train)
#========
test = fillna_obj(nan_obj_cols_test , test)

In [514]:
def a2int(obj_cols , train):
    size = len(obj_cols)
    
    for i in range(size):
        train[obj_cols[i]] = train[obj_cols[i]].astype(int)
    
    return train

In [515]:
#非数值转为数值后再将类型转为int
train = a2int(obj_cols , train)
#==========
test = a2int(obj_cols_test , test)

In [516]:
#已经完成对非数值的全部转换
#接下来便是 数值feature的转化 可以使用分阶段转化

In [517]:
def get_num_feature(train):
    cols = train.columns
    size = len(cols)
    
    num_feature = []
    
    for i in range(size):
        if train[cols[i]].dtype == np.int64 or train[cols[i]].dtype == np.float64:
            num_feature.append(cols[i])
    
    return num_feature

In [518]:
#数值类型的feature
num_feature = get_num_feature(train)
#=======
num_feature_test = get_num_feature(test)

In [519]:
print(num_feature)
#===========
print(num_feature_test)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'MoSold', 'YrSold']
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'MoSold', 'YrSold']


In [520]:
#因为这三个比较特殊
#都是年份
#YearBuilt YearRemodAdd GarageYrBlt YrSold 三个的数值化
def year_map(num):
    if num < 1900:
        return 12
    else:
        if num >= 2000:
            return 11
        else:
            return round(num%100/10 , 0) + 1

def year_map_z(num):
    return num % 100

In [521]:
train['YearBuilt'] = train['YearBuilt'].apply(year_map).astype(int)
train['YearRemodAdd'] = train['YearRemodAdd'].apply(year_map).astype(int)
train['GarageYrBlt'] = train['GarageYrBlt'].apply(year_map).astype(int)
train['YrSold'] = train['YrSold'].apply(year_map_z).astype(int)

#===========
test['YearBuilt'] = test['YearBuilt'].apply(year_map).astype(int)
test['YearRemodAdd'] = test['YearRemodAdd'].apply(year_map).astype(int)
test['GarageYrBlt'] = test['GarageYrBlt'].apply(year_map).astype(int)
test['YrSold'] = test['YrSold'].apply(year_map_z).astype(int)

In [522]:
#MSSubClass处理
def mss_map(num):
    return round(num/10 , 0)

In [523]:
train['MSSubClass'] = train['MSSubClass'].apply(mss_map).astype(int)
train['LotFrontage'] = train['LotFrontage'].apply(mss_map).astype(int)
#========
test['MSSubClass'] = test['MSSubClass'].apply(mss_map).astype(int)
test['LotFrontage'] = test['LotFrontage'].apply(mss_map).astype(int)

In [524]:
#临时保存一下
temp = train

tem_test = test

In [525]:
#恢复一下
train = temp

In [526]:
num_feature_no_trans = ['LotArea','MasVnrArea', 'BsmtFinSF1',
                        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                        'GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF']

num_feature_no_trans_test = ['LotArea','MasVnrArea', 'BsmtFinSF1',
                        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                        'GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF']

In [527]:
i = 0

def extra_map(num):
    global i
    
    _ = pd.qcut(train[num_feature_no_trans[i]] , 8)
    __ = _.unique().sort_values()
    
    size = len(__)
    for j in range(size):
        if num in __[j]:
            return j+1

i_test = 0

def extra_map_test(num):
    global i_test
    
    _ = pd.qcut(test[num_feature_no_trans_test[i_test]] , 8)
    __ = _.unique().sort_values()
    
    size = len(__)
    #print(num , end=',')
    for j in range(size):
        if num in __[j]:
            return j+1

In [528]:
#train 数据的转化
for m in [0,3,4,5,7,8]:
    i = m
    train[num_feature_no_trans[m]] = train[num_feature_no_trans[m]].apply(extra_map).astype(int)

In [529]:
i_test=0
test[num_feature_no_trans_test[0]] = test[num_feature_no_trans_test[0]].apply(extra_map_test).astype(int)

In [530]:
i_test=3
test[num_feature_no_trans_test[3]] = test[num_feature_no_trans_test[3]].apply(extra_map_test).astype(int)

In [531]:
i_test=4
test[num_feature_no_trans_test[4]] = test[num_feature_no_trans_test[4]].apply(extra_map_test).astype(int)

In [532]:
i_test=5
test[num_feature_no_trans_test[5]] = test[num_feature_no_trans_test[5]].apply(extra_map_test).astype(int)

In [533]:
i_test=7
test[num_feature_no_trans_test[7]] = test[num_feature_no_trans_test[7]].apply(extra_map_test).astype(int)

In [534]:
i_test=8
test[num_feature_no_trans_test[8]] = test[num_feature_no_trans_test[8]].apply(extra_map_test).astype(int)

In [535]:
ii = 0

def extra_map_cut(num):
    global ii
    
    _ = pd.cut(train[num_feature_no_trans[ii]] , 8)
    __ = _.unique().sort_values()
    
    size = len(__)
    for j in range(size):
        if num in __[j]:
            return j+1

ii_test = 0

def extra_map_cut_test(num):
    global ii_test
    
    _ = pd.cut(test[num_feature_no_trans_test[ii_test]] , 8)
    __ = _.unique().sort_values()
    
    size = len(__)
    for j in range(size):
        if num in __[j]:
            return j+1

In [536]:
for m in [1,2,6,9,10]:
    ii = m
    train[num_feature_no_trans[m]] = train[num_feature_no_trans[m]].apply(extra_map_cut).astype(int)
#===========
for m in [1,2,6,9,10]:
    ii_test=m
    test[num_feature_no_trans_test[m]] = test[num_feature_no_trans_test[m]].apply(extra_map_cut_test).astype(int)

In [537]:
train_y['SalePrice'] = train_y['SalePrice'].apply(lambda x:np.log(x))

In [416]:
def use(function = SVR):
    model = function()
    model.fit(train[0:1200] , train_y[0:1200])
    
    score = round(model.score(train[1200:] , train_y[1200:]) , 3)
    
    print(score)
    
    return model.predict(train[1200:])

In [121]:
use(DecisionTreeRegressor)

0.663


array([ 11.74403719,  12.27139211,  11.8493977 ,  12.36307639,
        11.91839057,  12.39462475,  11.35040654,  12.07254125,
        11.6307085 ,  12.65235975,  12.18075484,  12.12323437,
        11.41861479,  11.37939407,  11.81672692,  11.68266824,
        11.74403719,  12.44508955,  11.51292546,  11.62625415,
        11.68266824,  11.69107165,  11.8493977 ,  11.86779879,
        12.10071213,  11.8493977 ,  12.18075484,  11.84222921,
        12.48420426,  11.44035477,  12.10071213,  11.7905572 ,
        11.40756495,  11.8492191 ,  11.55214618,  11.53272809,
        12.01188092,  12.27839331,  12.13350195,  12.3863087 ,
        12.24769432,  12.26857785,  12.02574909,  12.79385931,
        12.3863087 ,  11.9511804 ,  12.19095901,  11.9316358 ,
        11.9511804 ,  11.81303006,  12.4049235 ,  12.15477935,
        11.51292546,  12.70076889,  12.04355372,  11.81303006,
        12.88664104,  11.35040654,  12.14950229,  11.88448902,
        12.15477935,  11.82041016,  11.8493977 ,  11.82

In [122]:
results = use(RandomForestRegressor)
np.exp(results)

0.833


I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


array([  99506.84912883,  223523.50178811,  115307.32186049,
        216576.43650175,  144313.67136502,  252840.22766667,
        118352.54387999,  188863.93917208,  141024.61492251,
        286771.6040064 ,  192933.01873047,  212529.71419276,
         85979.36702985,   85796.203993  ,  134351.7607146 ,
        113913.04952523,  167652.16869428,  249947.87347121,
         81993.69957503,  106093.77630295,  114947.65731165,
        128572.86328571,  135504.595291  ,  133883.57945387,
        197373.94473987,  134866.67644708,  206802.1169516 ,
        133874.62513385,  357413.26305884,  141037.01219202,
        189063.5929287 ,  143352.39189604,  113533.65468544,
        149676.75572697,  131116.51319441,  140733.2543858 ,
        166248.33831375,  214797.92783669,  161717.49168176,
        251894.51758678,  203936.44882481,  235282.08013712,
        176273.20350711,  374097.19962701,  206845.30840808,
        195405.6333766 ,  200965.241327  ,  150195.30970364,
        178325.73267154,

In [438]:
kf = KFold(train.shape[0] , n_folds=3 , random_state=1)
model = RandomForestRegressor()

for x ,vali in kf:
    model.fit(train.iloc[x , :] , train_y.iloc[x])
    
    print('score:' , round(model.score(train.iloc[vali , :] , train_y.iloc[vali]) , 3))

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


score: 0.839
score: 0.832
score: 0.841


I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [420]:
model3 = RandomForestRegressor()
model3.fit(train , train_y)

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [421]:
test_results = model3.predict(test)

In [426]:
test_results = np.exp(test_results)

In [427]:
idd = [x+1461 for x in range(1459)]

In [431]:
submit = pd.DataFrame({'Id':idd , 'SalePrice':test_results})

In [432]:
submit.to_csv('my_result.csv' , index=False)

In [433]:
#=================
#=================
#=================
#用神经网络train一发

In [538]:
len(train.columns)

39

In [539]:
xs = tf.placeholder(tf.float32 , shape = [None , 39])
ys = tf.placeholder(tf.float32 , shape = [None , 1])

ys = tf.nn.sigmoid(ys)

w1 = tf.Variable(tf.random_normal([39,20]) , dtype=tf.float32)
b1 = tf.Variable(tf.zeros([20]) , dtype=tf.float32)

w2 = tf.Variable(tf.random_normal([20,5]) , dtype=tf.float32)
b2 = tf.Variable(tf.zeros([5]) , dtype=tf.float32)

w3 = tf.Variable(tf.random_normal([5,1]) , dtype=tf.float32)
b3 = tf.Variable(tf.zeros([1]) , dtype=tf.float32)

l1 = tf.nn.sigmoid(tf.matmul(xs , w1) + b1)
l2 = tf.nn.sigmoid(tf.matmul(l1 , w2) + b2)
l3 = tf.nn.sigmoid(tf.matmul(l2 , w3) + b3)

loss = tf.reduce_mean(tf.reduce_sum(tf.abs(l3 - ys)))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [540]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

start = 0
batch_size = 100

for i in range(100):
    x_train = train.iloc[start :start+batch_size , :]
    y_train = np.reshape(train_y.iloc[start:start+batch_size , 0] , (batch_size , 1))
    
    print(sess.run(loss , feed_dict={xs:x_train , ys:y_train}))
    
    start += batch_size
    
    if start > 1200:
        start = 0

I:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67
1157.84
1155.31
1159.25
1159.77
1157.22
1166.0
1161.78
1162.25
1157.55
1160.71
1160.7
1165.41
1159.67


In [470]:
y_train = train_y.iloc[start:start+batch_size , 0]

In [474]:
y_train.shape

(100, 1)